In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta


pickle_path=os.path.join('..', 'data', 'features_by_date')
features_by_date=pd.read_pickle(pickle_path)
features_by_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 2022-03-09 to 2022-11-25
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   date                                     180 non-null    object 
 1   num_people_10_00                         180 non-null    int64  
 2   label_num_people_12_33                   180 non-null    int64  
 3   label_num_people_11_30                   180 non-null    int64  
 4   label_num_menus_sold                     180 non-null    float64
 5   label_difference_12_normalized           180 non-null    float64
 6   label_difference_10_and_sold_normalized  180 non-null    float64
 7   zurich_vacation                          180 non-null    int32  
 8   weekday                                  180 non-null    int64  
 9   Monday                                   180 non-null    int64  
 10  Tuesday                                

In [6]:
    pickle_path = os.path.join('..', 'data', 'flow_counts_preprocessed')
    df = pd.read_pickle(pickle_path)
    weekdays = [0, 1, 2, 3, 4]  # mon, tue, wed, thu, fri
    date_list_all = list(df[(df['time_numeric'] == 12.33)]['date'])
    date_list_weekdays = [date for date in date_list_all if date.weekday() in weekdays]
    date_list_weekends_and_holidays = list(df[(df['time_numeric'] == 12.33) & (df['num_people_inside'] < 25)]['date'])
    date_list_holidays = [date for date in date_list_weekends_and_holidays if date.weekday() in weekdays]
    date_list_workdays = list(df[(df['time_numeric'] == 12.33) & (df['num_people_inside'] > 25)]['date'])
    weekday_date_lists = {}
    for i in weekdays:
        weekday_date_lists[i] = [date for date in date_list_workdays if date.weekday() == i]

    monday_list=weekday_date_lists[0]

    #baseline with num people at 10 as only feature
    feature_list=['label_num_people_12_33','num_people_10_00']
    data=features_by_date.loc[monday_list,feature_list]

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

train, test = train_test_split(data)

train_x = train.drop(["label_num_people_12_33"], axis=1)
test_x = test.drop(["label_num_people_12_33"], axis=1)
train_y = train[["label_num_people_12_33"]]
test_y = test[["label_num_people_12_33"]]

alpha = 0.1
l1_ratio = 0.5

lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)

predicted_qualities = lr.predict(test_x)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)
print(f'Intercept: {lr.intercept_} \n Coefficients: {lr.coef_}')

Elasticnet model (alpha=0.100000, l1_ratio=0.500000):
  RMSE: 5.609773151748578
  MAE: 4.48513027496059
  R2: 0.9822215825305108
Intercept: [11.53130303] 
 Coefficients: [0.88099627]


In [15]:
def func(x=1,y=2):
    return x*y
x_y_dic= {'x':5, 'y':4}
func(**x_y_dic)

20

In [18]:
x_y_list=[3,4]
func(*x_y_list)

12

In [19]:
max([3,4,2])

4